In [2]:
import cv2
import argparse
import numpy as np
import utils
import os 
import time

In [2]:
%%time
utils.extract_frames("/data/TrainSet/Drone1/Morning/1.1.1.mov", out_dir='/data/out', save=True, skip=25)

Frames saved to `/data/out/` directory.
CPU times: user 3min 55s, sys: 5.85 s, total: 4min 1s
Wall time: 2min 46s


In [5]:
%%time
# read input image
list_images = utils.get_files_by_ext("/data/out", "jpg")
print("We got ", len(list_images), " images")

We got  180  images
CPU times: user 14.8 ms, sys: 10.5 ms, total: 25.3 ms
Wall time: 906 ms


In [8]:
%%time
classes_file = "object-detection-opencv/yolov3.txt"
weights_file = "object-detection-opencv/yolov3.weights"
conf_file = "object-detection-opencv/yolov3.cfg"

conf_threshold = 0.5
nms_threshold = 0.4

# for image_path in list_images:
image = cv2.imread(list_images[0])

Width = image.shape[1]
Height = image.shape[0]

scale = 0.00392

# read class names from text file
classes = None
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
# reads the weights and config file and creates the network.
net = cv2.dnn.readNet(weights_file, conf_file)

batch_size = 10
batch_images = list()

# for i in range(len(list_images)):
for i in range(10, 20):
    batch_images.append(list_images[i])
    if i == len(list_images)-1 or len(batch_images)%batch_size==0:      
        
        time_min_blobing = time.time()
        # create input blob 
        # prepares the input image to run through the deep neural network
        blob = utils.image_to_matrix(batch_images)

        # set input blob for the network
        net.setInput(blob)
        time_max_blobing = time.time()
        elapsed_time_blobing = time_max_blobing - time_min_blobing
        print("The blobing phase took ", round(elapsed_time_blobing,2), "s to be executed")
        print("\n")
        
        
        time_min_detection = time.time()
        # run inference through the network
        # and gather predictions from output layers
        outs = net.forward(utils.get_output_layers(net))

        # identify objects into the list of images analyzed by the network
        dict_obj_detected = {}
        dict_obj_detected = utils.detect_object(outs, batch_images, Width, Height)

        # identify pictures with at least one perso
        dict_danger = utils.detect_danger(dict_obj_detected)
        for image_name, is_dangerous in dict_danger.items():
            print("Is", image_name, "dangerous ? \t\t\t" , is_dangerous)
        time_max_detection = time.time()
        elapsed_time_detection = time_max_detection - time_min_detection
        print("The detection took ", round(elapsed_time_detection,2), "s to be executed")
        
        time_min_draw_bbox = time.time()
        # Draw bounding boxes and save images
        for image_path, image_items in dict_obj_detected.items():
            utils.get_bounding_box(image_path, image_items, \
                                   classes, COLORS, conf_threshold, nms_threshold
                                  )
        time_max_draw_bbox = time.time()
        elapsed_time_draw_bbox = time_max_draw_bbox - time_min_detection
        print("The draw of the bounding boxes and the image save took ", round(elapsed_time_draw_bbox,2), "s to be executed")         
        print("---------------------------------------------------------------------")
        
        batch_images = list()
    


The blobing phase took  1.51 s to be executed


3
(10, 507, 85)
(10, 2028, 85)
(10, 8112, 85)
Is /data/out/1.1.1.mov_frame00250.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00275.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00300.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00325.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00350.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00375.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00400.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00425.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00450.jpg dangerous ? 			 True
Is /data/out/1.1.1.mov_frame00475.jpg dangerous ? 			 True
The detection took  12.81 s to be executed
The draw of the bounding boxes and the image save took  19.0 s to be executed
---------------------------------------------------------------------
CPU times: user 28.8 s, sys: 486 ms, total: 29.3 s
Wall time: 23 s


In [3]:
utils.convert_frames_to_video("/data/out/output_with_bounding_box/", "/data/out/output_video/video.mp4", 5)

100%|██████████| 90/90 [04:28<00:00,  2.99s/it]
